### -------------Yanolja crawling---------------

In [1]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd
import json

### 1. scrapy로 룸정보 제외한 기본정보 가져오기

In [142]:
# 1. 프로젝트 생성

In [14]:
!rm -rf yanolja_hotel
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [15]:
!tree yanolja_hotel

yanolja_hotel
├── scrapy.cfg
└── yanolja_hotel
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py
    └── spiders
        └── __init__.py

2 directories, 7 files


In [16]:
# 2. xpath 찾기 : headers 설정

In [2]:
import requests #호텔 key값 얻는 모듈 생성 -> yanolja.py
import datetime

def get_hotel_keys():    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36", 
           "X-Requested-With": "XMLHttpRequest"
              }
    today = datetime.date.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    region = "서울"
    num=0
    page = 1
    key_info = []
    while True:
        parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
    &checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
    searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        #print('page=', str(apage))
        page += 1

        if response["motels"]["counts"] > 0:
            key = response["motels"]
            key_info.append(key)
        else:
            break
    hotel_keys = []
    for i in key_info:
        for j in i['lists']:
            hotel_keys.append(j['key'])        
    return hotel_keys

In [3]:
hotel_keys = get_hotel_keys()
len(hotel_keys)

416

### ------------xpath 확인 -> df확인 --------------

In [4]:
links = []
url = "https://www.yanolja.com/hotel/"
for hotel_key in hotel_keys:
    hotel_url = url + hotel_key
    links.append(hotel_url)
len(links)

416

In [5]:
link = links[0]
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.yanolja.com/hotel/3013410>

In [6]:
import json
location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
flatform = response.url.split(".")[1]
location, flatform

('서울특별시 영등포구 경인로 870', 'yanolja')

In [14]:
locations, names, levels, scores, review_counts, room_types, room_details, prices_1, loading_urls = [], [], [], [], [] ,[], [], [], []

num1 = 0
for link in links[:10]:
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        req = requests.get(link, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        try:
            price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
            prices=[]
            for num in range(len(room_type)):
                price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
                price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
                prices.append(price)
        except:
            prices= ["예약마감"]
    except:
        location = "페이지 없음"
        location = "페이지 없음"
        name = "페이지 없음"
        level = "페이지 없음"
        score = "페이지 없음"
        review_count = "페이지 없음"
        room_type = ["페이지 없음"]
        room_detail = ["페이지 없음"]
        price = ["페이지 없음"]
        prices= ["페이지 없음"]
        
    location = [location] * len(room_type)
    name = [name] * len(room_type)
    level = [level] * len(room_type)
    score = [score] * len(room_type)
    review_count = [review_count] * len(room_type)
    link = [link] * len(room_type)
    
    locations.extend(location)
    names.extend(name)
    levels.extend(level)
    scores.extend(score)
    review_counts.extend(review_count)
    room_types.extend(room_type)
    room_details.extend(room_detail)
    prices_1.extend(prices)
    loading_urls.extend(link)
    num1+=1
    print(str(num1) ,name[0])

1 페어필드 바이 메리어트 서울
2 나인트리 프리미어 명동2
3 신라스테이 구로
4 영등포 GMS 호텔
5 글래드 마포
6 노보텔 앰배서더 서울 용산
7 신라스테이 마포
8 홀리데이인 익스프레스 서울 홍대
9 더 파크호텔 서울
10 호텔 더 디자이너스 여의도


In [76]:
df = pd.DataFrame({
    'Name': names, 'Level': levels, 'Score':scores, 'Location': locations,
    'RoomType': room_types,'Roomdetail': room_details ,'Price': prices_1, 'Link': loading_urls, 'Review' : review_counts,
})
df.tail()

,Name,Level,Score,Location,RoomType,Roomdetail,Price,Link,Review
3388,종로 호텔 팝,비즈니스,5.0,서울특별시 종로구 수표로 22길 16,디럭스 더블,주차가능/넷플릭스,"62,360원",https://www.yanolja.com/hotel/1000111306,1
3389,종로 호텔 팝,비즈니스,5.0,서울특별시 종로구 수표로 22길 16,프리미어 테라스 더블,주차가능/넷플릭스,"74,120원",https://www.yanolja.com/hotel/1000111306,1
3390,종로 호텔 팝,비즈니스,5.0,서울특별시 종로구 수표로 22길 16,스위트 테라스 더블,주차가능/넷플릭스,"85,890원",https://www.yanolja.com/hotel/1000111306,1
3391,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,https://www.yanolja.com/hotel/1000109348,페이지 없음
3392,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,https://www.yanolja.com/hotel/1000112890,페이지 없음


### --------확인 끝 spider.py로 확장---------

In [19]:
# 3. items.py : title, content, link, category

In [2]:
%%writefile yanolja_hotel/yanolja_hotel/items.py
import scrapy

class YanoljaHotelItem(scrapy.Item):
    location = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    link = scrapy.Field()
    room_type = scrapy.Field()
    flatform = scrapy.Field()

Overwriting yanolja_hotel/yanolja_hotel/items.py


In [3]:
# 4. spider.py

In [11]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja_hotel.items import YanoljaHotelItem
from yanolja import get_hotel_keys
import json
import datetime

class Spider(scrapy.Spider):
    name = "YanoljaHotel"
    allow_domain = ["https://www.yanolja.com"]
    start_urls = ["https://www.yanolja.com/hotel/"]
    
    def parse(self, response):
        hotel_keys = get_hotel_keys()
        start_urls = "https://www.yanolja.com/hotel/"
        links = []
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        for hotel_key in hotel_keys:
            hotel_url = start_urls + hotel_key
            links.append(hotel_url)
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    def parse_content(self, response):
        item = YanoljaHotelItem()
        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        item["location"] = location
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        item["name"] = name
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        item["level"] = level
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        item["score"] = score
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        item["review_count"] = review_count
        link = response.url
        item["link"] = link
        flatform = response.url.split(".")[1]
        item["flatform"] = flatform
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
            item["room_type"] = room_type
        else:    
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
            item["room_type"] = room_type
        yield item
        

Overwriting yanolja_hotel/yanolja_hotel/spiders/spider.py


In [12]:
# 5. settings.py : robots.txt 여부 파악 

In [13]:
# 6. 프로젝트 실행

In [14]:
%%writefile run.sh
cd yanolja_hotel
rm yanolja_hotel.csv
scrapy crawl YanoljaHotel -o yanolja_hotel.csv

Overwriting run.sh


In [15]:
!/bin/bash run.sh

2021-03-09 13:16:41 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: yanolja_hotel)
2021-03-09 13:16:41 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-09 13:16:41 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-09 13:16:41 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'yanolja_hotel',
 'NEWSPIDER_MODULE': 'yanolja_hotel.spiders',
 'SPIDER_MODULES': ['yanolja_hotel.spiders']}
2021-03-09 13:16:41 [scrapy.extensions.telnet] INFO: Telnet Password: f69b892f557a0385
2021-03-09 13:16:41 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

2021-03-09 13:16:46 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-09&checkoutDate=2021-03-10&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=12&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-09 13:16:46 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.yanolja.com:443
2021-03-09 13:16:46 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-09&checkoutDate=2021-03-10&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=13&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-09 13:16:46 [urllib3.connectionpool] DEBUG: Sta

2021-03-09 13:16:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/1000102261> (referer: https://www.yanolja.com/hotel/)
2021-03-09 13:16:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001791> (referer: https://www.yanolja.com/hotel/)
2021-03-09 13:16:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009495>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3009495',
 'location': '서울특별시 마포구 홍익로5길 32',
 'name': '홍대 나인브릭 호텔',
 'review_count': '1,461',
 'room_type': ['디럭스 더블',
               '슈페리어 트윈 + 레이트체크아웃',
               '슈페리어 더블 + 레이트체크아웃',
               '슈페리어 트윈',
               '슈페리어 더블',
               '디럭스 더블 + 레이트체크아웃',
               '슈페리어 트윈',
               '슈페리어 더블',
               '디럭스 더블',
               '슈페리어 트윈',
               '슈페리어 더블',
               '디럭스 더블',
               '디럭스 더블',
               '주니어 펜트하우스',
               '주니어 펜트하우스',
 

2021-03-09 13:16:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000690>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/3000690',
 'location': '서울특별시 강남구 봉은사로 410',
 'name': '라마다 서울 강남',
 'review_count': '2,581',
 'room_type': ['패밀리 트윈',
               '슈페리어 트윈',
               '슈페리어 더블',
               '패밀리 트윈',
               '슈페리어 트윈',
               '슈페리어 더블',
               '슈페리어 더블',
               '슈페리어 트윈',
               '슈페리어 트윈',
               '패밀리 트윈',
               '슈페리어 트윈',
               '슈페리어 트윈',
               '패밀리 트윈',
               '슈페리어 더블',
               '슈페리어 더블',
               '패밀리 트윈',
               '슈페리어 트윈',
               '패밀리 트윈',
               '슈페리어 더블',
               '이그제큐티브 더블',
               '슈페리어 트윈',
               '슈페리어 더블',
               '이그제큐티브 트윈',
               '패밀리 트윈',
               '프리미어 더블',
               '패밀리 트윈',
               '패밀리 트윈',
               '슈페리어 트윈

2021-03-09 13:16:50 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/1000112780> (referer: https://www.yanolja.com/hotel/)
2021-03-09 13:16:50 [scrapy.core.scraper] ERROR: Spider error processing <GET https://www.yanolja.com/hotel/3007477> (referer: https://www.yanolja.com/hotel/)
Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/defer.py", line 120, in iter_errback
    yield next(it)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/python.py", line 353, in __next__
    return next(self.data)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/python.py", line 353, in __next__
    return next(self.data)
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/core/spidermw.py", line 62, in _evaluate_iterable
    for r in iterable:
  File "/home/ubuntu/.

...

2021-03-09 13:17:08 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3007173>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3007173',
 'location': '서울특별시 강서구 마곡중앙로 161-11',
 'name': '라마다 앙코르 마곡',
 'review_count': '1,361',
 'room_type': ['스탠다드 트윈', '스탠다드 더블', '패밀리 트윈', '주니어 스위트'],
 'score': '4.6'}
2021-03-09 13:17:08 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001104> (referer: https://www.yanolja.com/hotel/)
2021-03-09 13:17:08 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016266>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3016266',
 'location': '서울특별시 강남구 도산대로 216',
 'name': '에이든 바이 베스트웨스턴 청담',
 'review_count': '1,301',
 'room_type': ['체크인 시 배정',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드→디럭스 더블',
               '디럭스 트윈',
               '디럭

2021-03-09 13:17:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001604>
{'flatform': 'yanolja',
 'level': '2성급',
 'link': 'https://www.yanolja.com/hotel/3001604',
 'location': '서울특별시 송파구 오금로11길 55-16',
 'name': '잠실2.4호텔',
 'review_count': '1,707',
 'room_type': ['스탠다드',
               '비즈니스 트윈',
               '디럭스 트윈',
               '디럭스 더블',
               '스위트',
               '로얄 스위트A',
               '로얄 스위트B',
               '펜트하우스 TOGETHER',
               '펜트하우스 FOREVER'],
 'score': '4.5'}
2021-03-09 13:17:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001542>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3001542',
 'location': '서울특별시 종로구 삼봉로 71',
 'name': '신라스테이 광화문',
 'review_count': '1,617',
 'room_type': ['직장인 힘내3 HOUR+ 체크인 시 배정/주차불가',
               '내맘대로 8시간 스테이-스탠다드',
               '스탠다드 트윈',
               '스탠다드 더블',
               '체크인 시 배정 + 14시 체크아웃',
            

2021-03-09 13:17:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/1000101862>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.com/hotel/1000101862',
 'location': '서울특별시 종로구 율곡로 180',
 'name': '오라카이 대학로 호텔',
 'review_count': '1,703',
 'room_type': ['스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈 + 일리커피 2인',
               '스탠다드 더블 + 일리 커피 2인',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '스탠다드 트윈',
               '스탠다드 더블',
               '레지던스 스위트',
               '주니어 스위트',
               '스탠다드 트윈',
               '스탠다드 더블'],
 'score': '4.8

2021-03-09 13:17:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3008569> (referer: https://www.yanolja.com/hotel/)
2021-03-09 13:17:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3001559> (referer: https://www.yanolja.com/hotel/)
2021-03-09 13:17:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.yanolja.com/hotel/3006931> (referer: https://www.yanolja.com/hotel/)
2021-03-09 13:17:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001348>
{'flatform': 'yanolja',
 'level': '비즈니스',
 'link': 'https://www.yanolja.com/hotel/3001348',
 'location': '서울특별시 영등포구 국회대로70길 8',
 'name': '여의도 M 호텔',
 'review_count': '2,092',
 'room_type': ['체크인 시 객실 배정',
               '체크인 시 객실 배정',
               '체크인 시 객실 배정',
               '스탠다드 더블',
               '슈페리어 트윈',
               '디럭스 더블',
               '슈페리어 패밀리',
               '스위트룸',
               '슈페리어 트리플'],
 'score': '4.8'}
2021-03-09 1

2021-03-09 13:17:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3015974>
{'flatform': 'yanolja',
 'level': '3성급',
 'link': 'https://www.yanolja.com/hotel/3015974',
 'location': '서울시 마포구 양화로 188',
 'name': '홀리데이인 익스프레스 서울 홍대',
 'review_count': '2,846',
 'room_type': ['Stay Safe for 10hours 체크인 시 배정',
               '스탠다드 퀸',
               '슈페리어 트윈',
               '슈페리어 퀸',
               '슈페리어 트윈',
               '슈페리어 퀸',
               '슈페리어 트윈',
               '슈페리어 퀸',
               '슈페리어 헐리웃베드',
               '패밀리',
               '슈페리어 퀸→체크인시배정',
               '슈페리어 헐리웃베드',
               '슈페리어 트윈 + 2인 조식',
               '슈페리어 퀸 + 2인 조식',
               '슈페리어 트윈+배달의민족1만원',
               '슈페리어 퀸+배달의민족1만원',
               '슈페리어 퀸  + 2인 조식 + 레이트체크아웃'],
 'score': '4.7'}
2021-03-09 13:17:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'flatform': 'yanolja',
 'level': '4성급',
 'link': 'https://www.yanolja.c

In [16]:
pd.read_csv('./yanolja_hotel/yanolja_hotel.csv')

,flatform,level,link,location,name,review_count,room_type,score
0,yanolja,4성급,https://www.yanolja.com/hotel/3013410,서울특별시 영등포구 경인로 870,페어필드 바이 메리어트 서울,"8,771","Premier Twin,Premier King,Superior Single,Prem...",4.7
1,yanolja,3성급,https://www.yanolja.com/hotel/3012015,서울특별시 강남구 테헤란로 610,글래드 강남 코엑스센터,"1,536","스탠다드 더블,스탠다드 더블,스탠다드 트윈,스탠다드 트윈,스탠다드 더블,스탠다드 더...",4.7
2,yanolja,4성급,https://www.yanolja.com/hotel/3000643,서울특별시 중구 마른내로 71,호텔 피제이 명동,"1,673","스탠다드 트윈,스탠다드 더블,스탠다드 트윈,스탠다드 더블,스탠다드→프리미어 + Br...",4.7
3,yanolja,3성급,https://www.yanolja.com/hotel/3009495,서울특별시 마포구 홍익로5길 32,홍대 나인브릭 호텔,"1,461","디럭스 더블,슈페리어 트윈 + 레이트체크아웃,슈페리어 더블 + 레이트체크아웃,슈페리...",4.7
4,yanolja,4성급,https://www.yanolja.com/hotel/1000102261,서울특별시 종로구 인사동길 49,나인트리 프리미어 호텔 인사동,"1,457","8 to 8 머무ROOM 체크인 시 배정,스탠다드 트윈,스탠다드 더블,스탠다드 트윈...",4.8
5,yanolja,비즈니스,https://www.yanolja.com/hotel/3001791,서울특별시 중구 장충단로13길 20,호텔 스카이파크 킹스타운 동대문점,"1,637","주차불가,스탠다드 더블,주차불가★체크인 시 배정,주차불가★스탠다드 더블,주차불가★스...",4.7
6,yanolja,4성급,https://www.yanolja.com/hotel/3011846,서울특별시 강남구 테헤란로 415,L7 강남 바이 롯데,"1,490","스탠다드 트윈,스탠다드 룸,스탠다드 트윈,스탠다드 더블,스탠다드 트윈,스탠다드 트윈...",4.8
7,yanolja,3성급,https://www.yanolja.com/hotel/3001395,서울특별시 강남구 언주로 517,신라스테이 역삼,"1,808","내맘대로 반캉스/4시간 STAY - 체크인 시 배정,내맘대로 반캉스/8시간 STAY...",4.7
8,yanolja,3성급,https://www.yanolja.com/hotel/3001397,경기도 화성시 노작로 161,신라스테이 동탄,"1,619","17-22시 당일이용/5시간 STAY-스탠다드 체크인시 배정,9-19시 당일이용-스...",4.7
9,yanolja,4성급,https://www.yanolja.com/hotel/3001075,서울특별시 영등포구 의사당대로 18,글래드 여의도,"1,788","디럭스 트윈,스탠다드 더블,디럭스 트윈,스탠다드 더블,디럭스 트윈 + 레이트체크아웃...",4.7


In [ ]:
# 7. pipeline.py : 크롤링한 데이터를 mysql에 저장